In [0]:
!pip install --upgrade openai
dbutils.library.restartPython()

Selecting cohort of Penn OBSERVER patients.

In [0]:
import os
from openai import OpenAI

In [0]:
directory = "/Volumes/biomedicalinformatics_analytics/dev_lab_johnson/swimcap/Penn OBSERVER/problem_lists/"

problem_lists = []
for file in os.listdir(directory):
    with open(os.path.join(directory, file), "r") as fp:
        problem_lists.append( fp.read() )

In [0]:
DATABRICKS_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

client = OpenAI(
    api_key=DATABRICKS_TOKEN,
    base_url="https://adb-2035410508966251.11.azuredatabricks.net/serving-endpoints"
)

response = client.chat.completions.create(
    model="meta_llama_v3_2_3b_instruct",
    messages=[
        {
            "role": "user",
            "content": "I have provided you with problem lists for a collection of patients. Each patient’s problem list is separated by \"---\". Label each patient as either \"probable Alzheimer's dementia (AD)\" or \"Healthy control\" based on this information." + "\n---\n".join(problem_lists)
        }
    ]
)

print(response.choices[0].message.content)